# 수정한 점

1. 데이터의 다양화를 위하여 이미지를 가로로 뒤집거나 회전시켜서 데이터 증강함
2. 노드 dropout 적용
3. 이전의 방법에서 validation_loss가 점점 증가하던 것을 막고자 training data의 일부(0.1~0.2)를 validation data로 사용하였음
4. (몇번 학습해보니 loss가 감소하는 추세를 보이길래)hidden_unit의 수를 늘렸음 / hidden_layer의 수를 4개로 늘렸음



## 1. STL10 데이터셋 클래스 정의
STL10 데이터셋을 학습 및 테스트 데이터로 로드하고, 파이토치 데이터로더를 통해 배치 단위로 데이터를 준비하는 작업을 수행   
해당 데이터셋을 모댈 학습에 사용할 수 있도록 전처리와 로더 생성까지 포함

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split


# 데이터 변환 정의
transform = transforms.Compose([
    # 데이터 증강
    transforms.RandomHorizontalFlip(),  # 랜덤 가로 뒤집기
    transforms.RandomRotation(10),     # 랜덤 회전


    transforms.ToTensor(),       # 이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])

# STL10 학습 및 테스트 데이터셋 다운로드
train_dataset = datasets.STL10(root='./data', split='train', transform=transform, download=True)
test_dataset = datasets.STL10(root='./data', split='test', transform=transform, download=True)

# Training 데이터를 Validation Set으로 나누기
train_indices, val_indices = train_test_split(range(len(train_dataset)), test_size=0.2, random_state=42)
train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(train_dataset, val_indices)

# DataLoader 생성 (64개의 배치로 나누고, 학습 데이터를 셔플)
batch_size = 64
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [21]:
from torch import nn

# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, n_layers, output_size, activation_function):
        """
        Flexible한 MLP Model
        Args:
            input_size: Input 차원
            hidden_sizes : List of hidden layer sizes.
            output_size: Output 차원
            activation_fn : 여러개 중 선택해보기
        """

        super(MLP, self).__init__()


        layers = []
        previous_size = input_size

        # Hidden layers 생성
        for _ in range(n_layers):
            layers.append(nn.Linear(previous_size, hidden_size))  # Linear layer
            layers.append(activation_function())  # Activation function

            layers.append(nn.Dropout(0.5))  # 드롭아웃

            previous_size = hidden_size

        # Output layer 생성
        layers.append(nn.Linear(previous_size, output_size))
        self.model = nn.Sequential(*layers) # layers라는 list에 넣었던 입력층의 결과들을 다시 Sequential 모델로 결합

    def forward(self, x):
        return self.model(x)

# STL10 이미지 크기: 96x96, 채널 3 (3채널 이미지를 펼침)
input_size = 96 * 96 * 3
hidden_size = 256 # hidden unit의 개수를 128에서 점점 줄여서 나가도 됨 -- 만약 줄인다면--> 특징 추출을 세밀화
n_layers = 4  # Hidden layer 개수
output_size = 10  # 클래스 개수


activation_function = nn.ReLU # 활성화 함수 설정

model = MLP(input_size, hidden_size, n_layers, output_size, activation_function)

In [31]:
from torch import optim

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# Optimizer 설계
lr = 0.0005

# 에포크 설정
epochs = 100
optimizer = optim.Adam(model.parameters(), lr)


# epoch 별 train loss / validation loss / accuracy loss 저장할 list --> 매번 출력하여 추이 파악
list_epoch = []
list_train_loss = []
list_val_loss = []
list_acc = []
list_acc_epoch = []




# Train
# 학습 함수 정의
def train_model(model, dataloader, val_loader, criterion, optimizer, epochs):
    train_losses = []  # Store losses for each epoch
    val_losses = []  # Store validation losses for each epoch
    accuracies = []  # Store accuracies for each epoch

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in dataloader:

            images = images.view(images.size(0), -1)  # 이미지를 1D 벡터로 변환
            optimizer.zero_grad()
            outputs = model(images)  # 모델에 입력

            loss = criterion(outputs, labels)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 파라미터 업데이트
            train_loss = train_loss + loss.item()

        train_loss = train_loss / len(dataloader)
        list_train_loss.append(train_loss)
        list_epoch.append(epoch)

        # validation loss
        val_loss = evaluate_loss(model, val_loader, criterion)
        list_val_loss.append(val_loss)

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")


# loss function 평가
def evaluate_loss(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)



# Evaluation
# 평가 함수 정의
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 평가 시에는 그래디언트 계산 비활성화
        for images, labels in dataloader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # 예측 값 추출
            correct = correct + (predicted == labels).sum().item()
            total = total + labels.size(0)
    accuracy = correct / total
    list_acc.append(accuracy)
    list_acc_epoch.append(len(list_epoch))
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return accuracy


In [32]:
# 모델 학습
train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

Epoch [1/2], Train Loss: 2.3025, Val Loss: 2.3042
Epoch [2/2], Train Loss: 2.3022, Val Loss: 2.3042


In [ ]:
# 모델 평가
accuracy = evaluate_model(model, test_loader)


# 결과물 기록
hidden_size = 256 / hidden layer = 4 / lr = 0.0005 / 100 epoch - Accuracy: 43.01%


# train_loss / val_loss / accuracy를 그래프로 찍어보자


In [ ]:
import matplotlib.pyplot as plt

def plot_training_results(epochs, train_losses, val_losses, accuracies):
    plt.figure(figsize=(12, 4))

    # Train Loss와 Val Loss 그래프
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss', color='blue')
    plt.plot(epochs, val_losses, label='Validation Loss', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    # Accuracy 그래프 (accuracies가 비어있지 않을 경우)
    if accuracies:
        plt.subplot(1, 2, 2)
        plt.plot(epochs, accuracies, label='Accuracy', color='green')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title('Training Accuracy')
        plt.legend()

    plt.tight_layout()
    plt.show()